In [1]:
import os
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool
import multiprocessing
import numpy as np
import shutil
import pickle as pkl
import gc

from scipy.signal import butter, lfilter, resample

import torch

import multiprocessing.pool as mpp


def istarmap(self, func, iterable, chunksize=1):
    """starmap-version of imap
    """
    if self._state != mpp.RUN:
        raise ValueError("Pool not running")

    if chunksize < 1:
        raise ValueError(
            "Chunksize must be 1+, not {0:n}".format(
                chunksize))

    task_batches = mpp.Pool._get_tasks(func, iterable, chunksize)
    result = mpp.IMapIterator(self._cache)
    self._taskqueue.put(
        (
            self._guarded_task_generation(result._job,
                                          mpp.starmapstar,
                                          task_batches),
            result._set_length
        ))
    return (item for chunk in result for item in chunk)


mpp.Pool.istarmap = istarmap

In [2]:
data_path = '/labs/hulab/stark_stuff/ppg_ecg_project/data/'

def load_sigs(path, placeholder):
    sig = np.load(path)
    ecg = sig[:, 0]
    ppg = sig[:, 1]
    return ecg, ppg

def load_sigs_resample(path, placeholder):
    sig = np.load(path)
    ecg = sig[:, 0]
    ppg = sig[:, 1]
    ecg = resample(ecg, 2400)
    ppg = resample(ppg, 2400)
    
    return ecg, ppg

In [3]:
# split = 'train'
# AF_paths = pkl.load(open(f'{data_path}/AF_v2/split_{split}.pkl', 'rb'))
# NSR_paths = pkl.load(open(f'{data_path}/NSR_v2/split_{split}.pkl', 'rb'))
# PVC_paths = pkl.load(open(f'{data_path}/PVC_v2/split_{split}.pkl', 'rb'))

In [4]:
# train_AF_ecg = []
# train_AF_ppg = []
# pool_args = [[p, None] for p in AF_paths]

# with Pool(47) as pool:
#     for ret in tqdm(pool.istarmap(load_sigs, pool_args), total=len(pool_args)):
#         train_AF_ecg.append(ret[0])
#         train_AF_ppg.append(ret[1])        
        

In [5]:

# train_AF_ecg = np.asarray(train_AF_ecg)
# train_AF_ppg = np.asarray(train_AF_ppg)
# print(train_AF_ecg.shape, train_AF_ppg.shape)

In [6]:
# np.save(f'{data_path}/AF_v2/{split}_ECG.npy', train_AF_ecg)
# np.save(f'{data_path}/AF_v2/{split}_PPG.npy', train_AF_ppg)

In [3]:
def save_sig_npys(data_path, folder, split):
    paths = pkl.load(open(f'{data_path}/{folder}/split_{split}.pkl', 'rb'))
    ecgs = []
    ppgs = []
    pool_args = [[p, None] for p in paths]

    with Pool(47) as pool:
        for ret in tqdm(pool.istarmap(load_sigs, pool_args), total=len(pool_args)):
            ecgs.append(ret[0])
            ppgs.append(ret[1])
    
    ecgs = np.asarray(ecgs)
    ppgs = np.asarray(ppgs)
    print(ecgs.shape, ppgs.shape)
    
    np.save(f'{data_path}/{folder}/{split}_ECG.npy', ecgs)
    np.save(f'{data_path}/{folder}/{split}_PPG.npy', ppgs)
    
    del ecgs, ppgs, paths
    gc.collect()

In [4]:
def save_resample_sig_npys(data_path, folder, split):
    paths = pkl.load(open(f'{data_path}/{folder}/split_{split}.pkl', 'rb'))
    ecgs = []
    ppgs = []
    pool_args = [[p, None] for p in paths]

    with Pool(47) as pool:
        for ret in tqdm(pool.istarmap(load_sigs_resample, pool_args), total=len(pool_args)):
            ecgs.append(ret[0])
            ppgs.append(ret[1])
    
    ecgs = np.asarray(ecgs)
    ppgs = np.asarray(ppgs)
    print(ecgs.shape, ppgs.shape)
    
    np.save(f'{data_path}/{folder}/{split}_ECG_resampled2400.npy', ecgs)
    np.save(f'{data_path}/{folder}/{split}_PPG_resampled2400.npy', ppgs)
    
    del ecgs, ppgs, paths
    gc.collect()

In [5]:
!ls /labs/hulab/

12lead_ECG_for_ESUS  Anaconda3-2022.05-Linux-x86_64.sh	PROMOTE EHR data
4TB-drive	     chengding_project_data		ruochen_project
AF_code		     Echo Data				stark_conda
AF_DATA		     EEG CBFV				stark_stuff
Alarm_Data	     log.txt				Stroke_AF
AlarmData_UCSF_UCLA  physionet.org			sullychen
anaconda3	     PPG_QUALITY			UCSF


In [6]:
save_sig_npys(data_path, 'AF_v5', 'train')

100%|██████████| 2757889/2757889 [32:24<00:00, 1418.36it/s] 


MemoryError: Unable to allocate array with shape (2757889, 7200) and data type float64

In [ ]:
save_resample_sig_npys(data_path, 'AF_v5', 'val')